In [6]:
%matplotlib inline
import numpy as np
import healpy as hp
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import ICRS, Galactic, FK4, FK5

In [7]:
# Coordinates at the galactic pole
c = SkyCoord(Galactic, l=0.*u.degree , b=90*u.degree)

In [14]:
c.icrs.ra

<Longitude 192.85947789477598 deg>

In [16]:
# Parameters we need to set

# Magnitudes of gray cloud extinction one should be able to see through
max_extinction = 1.0

# Precision of the measured extinction
extinction_precision = 0.2

# Dynamic range of all sky camera
mag_range = [7, 12]

# Angular resolution requirement, diameter
ang_resolution = 3. #  Degrees

# Time resolution
time_resolution = 30. #  Seconds

# Seeing requirement

# Sky brightness requirement

# Airmass requrement
airmass_limit = 2.0

# Atmospheric extinction
kAtm = 0.15

In [15]:
# Querry for stars near the galactic pole, in the correct mag range and resolution diameter

In [17]:
# Zeropoint uncertainty is weighted mean of zeropoints computed from each star.  
# Should be able to adjust 5-sigma depth to get to extinction precision required. 